In [2]:
pip install torchvision --user

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opacus

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

In [5]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
               train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,)),]),),
               batch_size=64, shuffle=True, num_workers=1,
               pin_memory=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist',
              train=False,
              transform=transforms.Compose([transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,)),]),),
              batch_size=1024, shuffle=True, num_workers=1,
              pin_memory=True)

In [6]:
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3),
                            torch.nn.ReLU(),
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Conv2d(16, 32, 4, 2), 
                            torch.nn.ReLU(), 
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Flatten(), 
                            torch.nn.Linear(32 * 4 * 4, 32), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [7]:
privacy_engine = PrivacyEngine(model, 
                               batch_size=64, 
                               sample_size=60000,  
                               alphas=range(2,32), 
                               noise_multiplier=1.3, 
                               max_grad_norm=1.0,)

privacy_engine.attach(optimizer)

C:\Users\am4111\Anaconda3\lib\site-packages\opacus\privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
C:\Users\am4111\Anaconda3\lib\site-packages\opacus\privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


In [8]:
def train(model, train_loader, optimizer, epoch, device, delta):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    losses = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(delta)
        
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta}) for α = {best_alpha}")
    
for epoch in range(1, 11):
    train(model, train_loader, optimizer, epoch, device="cpu", delta=1e-5)

  0%|                                                                                          | 0/938 [00:00<?, ?it/s]C:\Users\am4111\Anaconda3\lib\site-packages\torch\nn\modules\module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

Train Epoch: 1 	Loss: 1.241036 (ε = 0.36, δ = 1e-05) for α = 22.0


  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

Train Epoch: 2 	Loss: 0.544475 (ε = 0.37, δ = 1e-05) for α = 22.0


  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

...

Train Epoch: 8 	Loss: 0.520670 (ε = 0.43, δ = 1e-05) for α = 22.0


  0%|                                                                                          | 0/938 [00:00<?, ?it/s]

Train Epoch: 9 	Loss: 0.522553 (ε = 0.44, δ = 1e-05) for α = 22.0


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:21<00:00, 43.55it/s]

Train Epoch: 10 	Loss: 0.550774 (ε = 0.45, δ = 1e-05) for α = 22.0


In [9]:
print(losses)

NameError: name 'losses' is not defined